In [1]:
!pip -q install openai langchain huggingface_hub transformers
!pip -q install tiktoken unstructured pdf2image pytesseract
!pip -q install chromadb
!pip install requests --use-feature=2.27.1
!pip install torch>=1.4 diffusers==0.10.2 sentencepiece jedi FLAX transformers scipy ftfy accelerate pyngrok==4.1.1 flask_ngrok datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

os.environ['OPENAI_API_KEY'] = ''
!ngrok authtoken ''


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
from langchain.document_loaders import UnstructuredExcelLoader
loader = UnstructuredExcelLoader('./sample_data/SydneyUni.xlsx', mode="elements")
data = loader.load()
data


[Document(page_content='\n  \n    \n      \n      \n      \n      \n      \n    \n    \n      \n      Degree\n      Course Requirements\n      Table A- Major or Minor\n      Table S\n    \n    \n      \n      Bachelor of Commerce\n      To qualify for the award of the Bachelor of Commerce, a candidate must complete 144 credit points, comprising:\\n\\n24 credit points of degree core units of study;\\na major (48 credit points) or program selected from the Table A subject areas;\\na minor (36 credit points) or second major (48 credit points) selected from Table A or Table S;\\n12 credit points of units of study in the Open Learning Environment (OLE) selected from Table O; and\\nany additional elective units of study from Tables A, S, or O required to make up the 144 credit point total\n      Accounting, Accounting (Professional), Banking, Business Analytics, Business Information Systems, Business Law, Finance, Industrial Relations and Human Resource Management, Innovation and Entrepreneu

In [4]:
from langchain.indexes import VectorstoreIndexCreator
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, render_template_string, request, jsonify

index = VectorstoreIndexCreator().from_loaders([loader])
# query = "What are the courses available on humanities?"
# index.query_with_sources(query)


# ===== Web Server with NgRok ===
app = Flask(__name__)
run_with_ngrok(app)


@app.route('/submit-prompt', methods=['POST'])
def generate():

    data = request.get_json()
    prompt = data.get('prompt', '')

    query = prompt
    response = index.query_with_sources(query)

    print("Sending image and text ...")
    data = {
        "response": response,
    }
    return jsonify(data)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f113-104-199-237-112.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/Jun/2023 10:13:41] "POST /submit-prompt HTTP/1.1" 200 -


Sending image and text ...
